In [ ]:
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import os
import sys
import cv2

In [ ]:

model = YOLO('yolo12n.pt') 

In [ ]:

dataset_path = Path('E:\Python\Data Science\Road Damage\pothole_detection_CNN\DatasetV12\data.yaml')
if not dataset_path.exists():
    print(f"Error: data.yaml not found at {dataset_path}")
    print("Please run prepare_dataset.py first")
    sys.exit(1)

In [ ]:
# Verify dataset structure
required_dirs = [
    'E:\\Python\\Data Science\\Road Damage\\pothole_detection_CNN\\DatasetV12\\images\\train',
    'E:\\Python\\Data Science\\Road Damage\\pothole_detection_CNN\\DatasetV12\\images\\val',
    'E:\\Python\\Data Science\\Road Damage\\pothole_detection_CNN\\DatasetV12\\images\\test',
    'E:\\Python\\Data Science\\Road Damage\\pothole_detection_CNN\\DatasetV12\\labels\\train',
    'E:\\Python\\Data Science\\Road Damage\\pothole_detection_CNN\\DatasetV12\\labels\\val',
    'E:\\Python\\Data Science\\Road Damage\\pothole_detection_CNN\\DatasetV12\\labels\\test'
]
for dir_path in required_dirs:
    if not Path(dir_path).exists():
        print(f"Error: Required directory {dir_path} does not exist")
        print("Please run prepare_dataset.py first")
        sys.exit(1)

In [ ]:
# Train the model
try:
    print("Starting training...")
    print(f"Using dataset at: {dataset_path.absolute()}")
    
    results = model.train(
        data=str(dataset_path),  # path to data.yaml file
        epochs=20,              # number of epochs
        imgsz=640,             # image size
        batch=16,              # batch size
        name='road_damage_detection',  # experiment name
        device='cpu'           # use CPU for training
    )
    print("Training completed successfully!")
    
except Exception as e:
    print(f"Error during training: {str(e)}")
    sys.exit(1)

In [ ]:
metrics = results.val()

In [ ]:
#Function to predict on new images
def predict_image(model, image_path):
    results = model(image_path)
    print(results)
    return results[0] 

In [ ]:
def plot_predictions(image_path, results):
    plt.figure(figsize=(10, 10))
    plt.imshow(results[0].plot())  # Access the first item in the results list
    plt.axis('off')
    plt.show()

In [ ]:

test_image_path = Path('E:\Python\Data Science\Road Damage\pothole_detection_CNN\DatasetV12\images\\test\\41_jpg.rf.d926b3ebc87c1cbb64bf2da4a893e9fa.jpg')
plot_predictions(test_image_path, results)

In [ ]:
if test_image_path.exists():
    results = predict_image(model, test_image_path)
    plot_predictions(test_image_path, results)
    for r in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = r
        print(f"Detected class {int(class_id)} with confidence {score:.2f}")
else:
    print(f"Test image not found at {test_image_path}") 

In [ ]:
# Load the trained model
model = YOLO("E:\Python\Data Science\Road Damage\pothole_detection_CNN\\runs\detect\\road_damage_detection4\weights\\best.pt")

# Predict on an image
results = model.predict("E:\Python\Data Science\Road Damage\pothole_detection_CNN\Test random data\p1.jpeg", save=False, show=False)
# Get the annotated image (with bounding boxes)
annotated_img = results[0].plot()  # returns a numpy array (BGR)

# Convert BGR to RGB for correct display in matplotlib
annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)

# Display it inline in Jupyter
plt.figure(figsize=(8, 5))
plt.imshow(annotated_img_rgb)
plt.axis('off')
plt.title("YOLOv8 Prediction - Pothole Detection")
plt.show()

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Load model
model = YOLO(r"E:\Python\Data Science\Road Damage\pothole_detection_CNN\runs\detect\road_damage_detection4\weights\best.pt")

# Predict on an image
img_path = r"E:\Python\Data Science\Road Damage\pothole_detection_CNN\Dataset\Surface Erosion\129.jpg"
results = model.predict(img_path, save=True, show=False)

# Check predictions
if results and results[0].boxes is not None:
    annotated_img = results[0].plot()
    annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(8, 5))
    plt.imshow(annotated_img_rgb)
    plt.axis('off')
    plt.title("YOLOv12 Prediction ")
    plt.show()
else:
    print("No detections found.")
